In [6]:
!nvidia-smi

Mon May  5 20:17:56 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.230.02             Driver Version: 535.230.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GH200 120GB             On  | 00000009:01:00.0 Off |                    0 |
| N/A   27C    P0              88W / 900W |     19MiB / 97871MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [7]:
import json
from pprint import pprint

import torch
from environs import env
from transformers import AutoModelForCausalLM, AutoTokenizer, QuantoConfig

from local_funcs import chat_funcs, prompt_funcs
from yiutils.project_utils import find_project_root

In [8]:
proj_root = find_project_root("justfile")
data_dir = proj_root / "data"

env.read_env(proj_root / ".env")

True

In [9]:
print(torch.cuda.is_available())
print(torch.version.cuda)

True
12.6


# data load

In [10]:
path_to_mr_pubmed_data = (
    data_dir / "intermediate" / "mr-pubmed-data" / "mr-pubmed-data.json"
)
assert path_to_mr_pubmed_data.exists(), (
    f"Data file {path_to_mr_pubmed_data} does not exist."
)

with open(path_to_mr_pubmed_data, "r") as f:
    mr_pubmed_data = json.load(f)

In [11]:
article_data = mr_pubmed_data[0]
article_data

{'pmid': '38794754',
 'ab': "Alcohol consumption significantly impacts disease burden and has been linked to various diseases in observational studies. However, comprehensive meta-analyses using Mendelian randomization (MR) to examine drinking patterns are limited. We aimed to evaluate the health risks of alcohol use by integrating findings from MR studies. A thorough search was conducted for MR studies focused on alcohol exposure. We utilized two sets of instrumental variables-alcohol consumption and problematic alcohol use-and summary statistics from the FinnGen consortium R9 release to perform de novo MR analyses. Our meta-analysis encompassed 64 published and 151 de novo MR analyses across 76 distinct primary outcomes. Results show that a genetic predisposition to alcohol consumption, independent of smoking, significantly correlates with a decreased risk of Parkinson's disease, prostate hyperplasia, and rheumatoid arthritis. It was also associated with an increased risk of chronic 

# model init

In [12]:
access_token = env("HUGGINGFACE_TOKEN")

## llama4

In [ ]:
# TODO
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# device = "cuda"
# dtype = torch.bfloat16
# tokenizer = AutoTokenizer.from_pretrained(model_id, token=access_token)
# quantization_config = QuantoConfig(weights="int4")
# model = AutoModelForCausalLM.from_pretrained(
#   model_id,
#   torch_dtype=dtype,
#   device_map=device,
#   token=access_token,
#   quantization_config=quantization_config,
# )

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.12s/it]


## deepseek

In [13]:
MODEL_ID = "DeepSeek-Prover-V2-7B"

ds_prover_tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
ds_prover_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True
)

OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like DeepSeek-Prover-V2-7B is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.